In [38]:
import pandas as pd
import subprocess
import json

MODEL = "llama3.2:1b"

def query_ollama(prompt, model=MODEL):
    """Send a prompt to Ollama and return its raw output."""
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    return result.stdout.decode("utf-8").strip()

def parse_row_with_llm(row: pd.Series):
    """Use Ollama to extract name and quantity from a DataFrame row."""
    # Convert row (Series) to dict for clarity
    row_dict = row.to_dict()

    prompt = f"""
You are a data extraction model. Interpret the following dictionary representing one row of a CSV file:

{json.dumps(row_dict)}

Your task:
- Identify which field in this row refers to the hardware part or item name.
- Identify which field refers to the numeric quantity.

Return a **single JSON object only**, with exactly these two keys:
  {{
    "name": "<part or item name, string>",
    "quantity": <integer or float quantity, set as integer in output>
  }}

Rules:
- Output only valid JSON (no markdown, no explanations, no code).
- If a field is missing, output `null` for that key.
"""
    raw = query_ollama(prompt)
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        parsed = {"name": None, "quantity": None}
    return pd.Series(parsed)

def process_csv(input_csv, output_csv="parsed_output.csv"):
    # Load CSV as DataFrame
    df = pd.read_csv(input_csv)
    results = []

    for _, row in df.iterrows():
        parsed = parse_row_with_llm(row)
        results.append(parsed)

    # Combine parsed results into DataFrame
    df_parsed = pd.DataFrame(results, columns=["name", "quantity"])
    df_parsed.to_csv(output_csv, index=False)
    print(f"✅ Parsed output saved to {output_csv}")

    return df_parsed

if __name__ == "__main__":
    process_csv("test_parts.csv")


✅ Parsed output saved to parsed_output.csv


In [37]:
import csv
import random

# possible header variations
possible_headers = [
    ["part_name", "qty"],
    ["item", "number_of_units"],
    ["hardware", "count"],
    ["component", "quantity"]
]
headers = random.choice(possible_headers)

# sample data pools
parts = [
    "Raspberry Pi 4", "Arduino Uno", "Breadboard", "Jumper Wires",
    "USB-C Cable", "12V Power Supply", "OLED Display", "Servo Motor",
    "DC Fan", "ESP32 Module", "LED Strip", "Heat Sink", "SD Card Reader",
    "Stepper Motor", "Relay Module"
]

# generate 1 random rows
rows = []
for _ in range(10):
    part = random.choice(parts)
    qty = random.randint(1, 50)
    rows.append([part, qty])

# write CSV
with open("test_parts.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(rows)

print(f"✅ Generated 'test_parts.csv' with columns: {headers}")

✅ Generated 'test_parts.csv' with columns: ['item', 'number_of_units']
